**I used papago for translation :)**

**이 노트북은 악성코드 데이터를 분석해보고 싶지만 너무 큰 데이터 용량에 가로막혀 분석을 진행하지 못하는 사람들을 위하여 제작되었습니다.**

**용량의 한계가 있어 asm 파일을 제외하고 byte 파일만 사용하였으며 byte 파일만을 이용하여 public leader board score에서 0.02742를 달성하였습니다.**

**byte파일만으로 데이터셋을 제작하여 용량에 제한을 받지 않고 따라할 수 있도록 하였습니다.**

**여러 방법을 시도한 다음, 모델을 평가하는 과정은 중복되는 부분이 많아 제거하고 성능이 좋은 부분만 남겼습니다.**

**용량의 제약이 없다면 asm 데이터를 사용하여 더 좋은 성능을 보이실 수 있습니다.**

**This code is designed for those who want to analyze malicious code data but are blocked by too much data capacity to proceed with the analysis.**

**Due to the capacity limitation, only byte files except for asm files were used, and 0.02742 was achieved at the public leader board core using only byte files.**

**We made a dataset with only byte files so that it can be followed without limitation in capacity.**

**After attempting several methods, the process of evaluating the model removed many overlapping parts and left only the good performance.**

**If you don't have a capacity limit, you can perform better using asm data.**

# **Import**

**pandas, numpy를 사용하고**

**진행상황을 알기 위하여 tqdm을 사용합니다.**

**파일을 읽기 위한 os와 경고 무시를 위한 warnings,**

**cv를 이용한 stacking data 제작을 위한 stratified kfold,**

**이외에 요즘 캐글에서 좋은 성능을 보이고 있는 boosting 모델들을 import 하였습니다.**

**use pandas, numpy**

**We use tqdm to know the progress.**

**os for reading files and warnings for ignoring warnings,**

**Stratified kfold for producing stacking data using cv,**

**In addition, we imported boosting models that are showing good performance in kaggle these days.**

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
fold = StratifiedKFold(n_splits=5, shuffle = True, random_state=62)

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# **데이터 불러오기(data load)**

**파일의 class를 적어놓은 trainlabels와 submission의 형식이 있는 sample submission을 읽어옵니다.**

**Read the sample submission in the form of train labels and submission where the class of the file is written.**

In [ ]:
train_labels=pd.read_csv("../input/malware-classification/trainLabels.csv")
sample = pd.read_csv("../input/malware-classification/sampleSubmission.csv",index_col="Id")
a = '"0","1","2","3","4","5","6","7","8","9","0a","0b","0c","0d","0e","0f","10","11","12","13","14","15","16","17","18","19","1a","1b","1c","1d","1e","1f","20","21","22","23","24","25","26","27","28","29","2a","2b","2c","2d","2e","2f","30","31","32","33","34","35","36","37","38","39","3a","3b","3c","3d","3e","3f","40","41","42","43","44","45","46","47","48","49","4a","4b","4c","4d","4e","4f","50","51","52","53","54","55","56","57","58","59","5a","5b","5c","5d","5e","5f","60","61","62","63","64","65","66","67","68","69","6a","6b","6c","6d","6e","6f","70","71","72","73","74","75","76","77","78","79","7a","7b","7c","7d","7e","7f","80","81","82","83","84","85","86","87","88","89","8a","8b","8c","8d","8e","8f","90","91","92","93","94","95","96","97","98","99","9a","9b","9c","9d","9e","9f","a0","a1","a2","a3","a4","a5","a6","a7","a8","a9","aa","ab","ac","ad","ae","af","b0","b1","b2","b3","b4","b5","b6","b7","b8","b9","ba","bb","bc","bd","be","bf","c0","c1","c2","c3","c4","c5","c6","c7","c8","c9","ca","cb","cc","cd","ce","cf","d0","d1","d2","d3","d4","d5","d6","d7","d8","d9","da","db","dc","dd","de","df","e0","e1","e2","e3","e4","e5","e6","e7","e8","e9","ea","eb","ec","ed","ee","ef","f0","f1","f2","f3","f4","f5","f6","f7","f8","f9","fa","fb","fc","fd","fe","ff","??","size","Class"'

# **전처리(preprocessing)**

```python
train_np = np.zeros((len(files),len(a.split(","))))
files = os.listdir("../input/malware-only-byte/only_byte")
files.sort()
k=0
for file in tqdm(files):
    statinfo=os.stat("../input/malware-only-byte/only_byte/"+file)
    with open("../input/malware-only-byte/only_byte/"+file,"r") as fp:
        for lines in fp.readlines():
            line=lines.rstrip().split(" ")[1:]
            for hex_code in line:
                if hex_code=='??':
                    train_np[k][256]+=1
                else:
                    train_np[k][int(hex_code,16)]+=1
        train_np[k][257]=statinfo.st_size/(1024*1024)
        train_np[k][258]=train_labels[train_labels["Id"]==file.split('.')[0]]["Class"].tolist()[0]
    fp.close()
    k += 1
train=pd.DataFrame(train_np, columns=a[1:-1].split('","'))
train.to_csv("train_data.csv",index=False)
train.head()
```

**위에 있는 코드는 파일명을 이용하여 파일을 읽어들이고 각 파일에 0,1,2,...,?? 등의 hex값의 수를 나타낸 코드입니다.**

**https://www.kaggle.com/paulrohan2020/microsoft-malware-detection-log-loss-of-0-0070 를 참고하였습니다.**

**실행시간이 길어서 markdown으로 적었습니다. 위 코드를 복사하시고 실행하실 수 있습니다.**

**The code above is a code that reads the file using the file name and represents the number of hex values such as 0, 1, 2, ..., ?? in each file.**

**I referred to https://www.kaggle.com/paulrohan2020/microsoft-malware-detection-log-loss-of-0-0070**

**I wrote down markdown because of the long execution time. You can copy and execute the code above.**

In [ ]:
train=pd.read_csv("../input/malware-only-byte/train_data.csv")

```python
file2=[]
for file in files:
    file2.append(file.split(".")[0])
test_np = np.zeros((len(files),len(a.split(","))))
files = os.listdir("../input/malware-only-byte/test")
files.sort()
k=0
for file in tqdm(files):
    statinfo=os.stat("../input/malware-only-byte/test/"+file)
    with open("../input/malware-only-byte/test/"+file,"r") as fp:
        for lines in fp.readlines():
            line=lines.rstrip().split(" ")[1:]
            for hex_code in line:
                if hex_code=='??':
                    test_np[k][256]+=1
                else:
                    test_np[k][int(hex_code,16)]+=1
        test_np[k][257]=statinfo.st_size/(1024*1024)
    fp.close()
    k += 1
test=pd.DataFrame(test_np, columns=a[1:-1].split('","'))
test["Id"] = file2
test.to_csv("test_data.csv",index=False)
test.head()
```

**위 코드는 test byte 파일에 대하여 진행한 코드입니다.**

**The code above is the code conducted for the test byte file.**

In [ ]:
test=pd.read_csv("../input/malware-only-byte/test_data.csv")

# **여러가지 model을 이용한 학습과 스태킹(modeling and stacking)**

In [ ]:
X = train.drop(['Class'], axis=1)
y = train['Class']

**예측할 값을 제외한 값을 X로, 예측값을 y로 나누어 줍니다.**

**Divide the values excluding the predicted values by X and y.**

**이후로 stacking을 위하여 train data에 cross validation을 진행하여 각 validation set에 대한 예측값으로 train data set의 예측값을 만듭니다.**

**바로 X,y를 학습시키고 예측을 진행해도 좋지만 stacking을 진행한 쪽이 더 성능이 좋았습니다.**

**After that, cross validation is performed on the train data for stacking to create a predicted value of the train data set as a predicted value for each validation set.**

**You can learn X and y right away and make predictions, but the one who did the stacking performed better.**

In [ ]:
stack_df = pd.read_csv("../input/malware-only-byte/train_data.csv")
for i,j in enumerate(fold.split(X,y)):
    stack_train_X = X.iloc[j[0]]
    stack_train_y = y.iloc[j[0]]
    stack_test_X = X.iloc[j[1]]
    stack_test_y = y.iloc[j[1]]
    model = LGBMClassifier(learning_rate= 0.025, n_estimators = 850, min_child_weight = 1, boosting_type = "gbdt", min_child_samples=68,random_state = 62,objective = "multi-class",metric = "multi_logloss")
    model.fit(stack_train_X,stack_train_y)
    preds = model.predict_proba(stack_test_X)
    preds = pd.DataFrame(preds)
    stack_df["0"].iloc[j[1]] = preds[0]
    stack_df["1"].iloc[j[1]] = preds[1]
    stack_df["2"].iloc[j[1]] = preds[2]
    stack_df["3"].iloc[j[1]] = preds[3]
    stack_df["4"].iloc[j[1]] = preds[4]
    stack_df["5"].iloc[j[1]] = preds[5]
    stack_df["6"].iloc[j[1]] = preds[6]
    stack_df["7"].iloc[j[1]] = preds[7]
    stack_df["8"].iloc[j[1]] = preds[8]
lgbm_stack = stack_df[["0","1","2","3","4","5","6","7","8"]]
lgbm_stack.columns = ["lgbm_Prediction1","lgbm_Prediction2","lgbm_Prediction3","lgbm_Prediction4","lgbm_Prediction5","lgbm_Prediction6","lgbm_Prediction7","lgbm_Prediction8","lgbm_Prediction9"]

In [ ]:
stack_df = pd.read_csv("../input/malware-only-byte/train_data.csv")
for i,j in enumerate(fold.split(X,y)):
    stack_train_X = X.iloc[j[0]]
    stack_train_y = y.iloc[j[0]]
    stack_test_X = X.iloc[j[1]]
    stack_test_y = y.iloc[j[1]]
    model = XGBClassifier(booster="gbtree",eta = 0.0975, min_child_weight = 2,random_state = 62, objective = "multi:softmax",eval_metric="logloss")
    model.fit(stack_train_X,stack_train_y)
    preds = model.predict_proba(stack_test_X)
    preds = pd.DataFrame(preds)
    stack_df["0"].iloc[j[1]] = preds[0]
    stack_df["1"].iloc[j[1]] = preds[1]
    stack_df["2"].iloc[j[1]] = preds[2]
    stack_df["3"].iloc[j[1]] = preds[3]
    stack_df["4"].iloc[j[1]] = preds[4]
    stack_df["5"].iloc[j[1]] = preds[5]
    stack_df["6"].iloc[j[1]] = preds[6]
    stack_df["7"].iloc[j[1]] = preds[7]
    stack_df["8"].iloc[j[1]] = preds[8]
xgb_stack = stack_df[["0","1","2","3","4","5","6","7","8"]]
xgb_stack.columns = ["xgb_Prediction1","xgb_Prediction2","xgb_Prediction3","xgb_Prediction4","xgb_Prediction5","xgb_Prediction6","xgb_Prediction7","xgb_Prediction8","xgb_Prediction9"]

In [ ]:
stack_df = pd.read_csv("../input/malware-only-byte/train_data.csv")
for i,j in enumerate(fold.split(X,y)):
    stack_train_X = X.iloc[j[0]]
    stack_train_y = y.iloc[j[0]]
    stack_test_X = X.iloc[j[1]]
    stack_test_y = y.iloc[j[1]]
    model = CatBoostClassifier(verbose=0)
    model.fit(stack_train_X,stack_train_y)
    preds = model.predict_proba(stack_test_X)
    preds = pd.DataFrame(preds)
    stack_df["0"].iloc[j[1]] = preds[0]
    stack_df["1"].iloc[j[1]] = preds[1]
    stack_df["2"].iloc[j[1]] = preds[2]
    stack_df["3"].iloc[j[1]] = preds[3]
    stack_df["4"].iloc[j[1]] = preds[4]
    stack_df["5"].iloc[j[1]] = preds[5]
    stack_df["6"].iloc[j[1]] = preds[6]
    stack_df["7"].iloc[j[1]] = preds[7]
    stack_df["8"].iloc[j[1]] = preds[8]
cat_stack = stack_df[["0","1","2","3","4","5","6","7","8"]]
cat_stack.columns = ["cat_Prediction1","cat_Prediction2","cat_Prediction3","cat_Prediction4","cat_Prediction5","cat_Prediction6","cat_Prediction7","cat_Prediction8","cat_Prediction9"]

In [ ]:
stacking_X = pd.concat([xgb_stack,lgbm_stack,cat_stack],axis=1)

**각 모델들을 이용하여 train set에 대하여 stacking을 진행한 다음 그 예측값을 새로운 학습 데이터로 만들었습니다.**

**Stacking was performed on the train set using each model, and the predicted value was made into new learning data.**

In [ ]:
model = LGBMClassifier(learning_rate= 0.025, n_estimators = 850, min_child_weight = 1, boosting_type = "gbdt", min_child_samples=68,random_state = 62,objective = "multi-class",metric = "multi_logloss")
model.fit(X,y)
lgbm_pred = model.predict_proba(test.drop("Id",axis=1))
lgbm_pred = pd.DataFrame(lgbm_pred)
lgbm_pred.columns = ["lgbm_Prediction1","lgbm_Prediction2","lgbm_Prediction3","lgbm_Prediction4","lgbm_Prediction5","lgbm_Prediction6","lgbm_Prediction7","lgbm_Prediction8","lgbm_Prediction9"]

In [ ]:
model = XGBClassifier(booster="gbtree",eta = 0.0975, min_child_weight = 2,random_state = 62, objective = "multi:softmax",eval_metric="logloss")
model.fit(X,y)
xgb_pred = model.predict_proba(test.drop("Id",axis=1))
xgb_pred = pd.DataFrame(xgb_pred)
xgb_pred.columns = ["xgb_Prediction1","xgb_Prediction2","xgb_Prediction3","xgb_Prediction4","xgb_Prediction5","xgb_Prediction6","xgb_Prediction7","xgb_Prediction8","xgb_Prediction9"]

In [ ]:
model = CatBoostClassifier(verbose=0)
model.fit(X,y)
cat_pred = model.predict_proba(test.drop("Id",axis=1))
cat_pred = pd.DataFrame(cat_pred)
cat_pred.columns = ["cat_Prediction1","cat_Prediction2","cat_Prediction3","cat_Prediction4","cat_Prediction5","cat_Prediction6","cat_Prediction7","cat_Prediction8","cat_Prediction9"]

In [ ]:
test_X = pd.concat([xgb_pred,lgbm_pred,cat_pred],axis=1)

**test data에 대한 예측값을 생성하였습니다.**

**A predicted value for test data was generated.**

In [ ]:
model = CatBoostClassifier(verbose=0)
model.fit(stacking_X,y)
pred = model.predict_proba(test_X)
imsi_df = pd.DataFrame(pred)
imsi_df = imsi_df.set_index(test["Id"])
submission = pd.concat([sample,imsi_df],axis=1)
submission = submission.drop(["Prediction1","Prediction2","Prediction3","Prediction4","Prediction5","Prediction6","Prediction7","Prediction8","Prediction9"],axis=1)
submission.columns = ["Prediction1","Prediction2","Prediction3","Prediction4","Prediction5","Prediction6","Prediction7","Prediction8","Prediction9"]
submission.to_csv("submission.csv")

**예측값을 기반으로 실제값과 학습시킨다음, test data의 예측값을 이용하여 예측을 진행하였습니다.**

**After learning the actual value based on the predicted value, the prediction was conducted using the predicted value of test data.**